#  Projeto de Roteirização Turística 

Este notebook implementa o projeto completo de recomendação turística com RAG, utilizando:
- `Router Chain` para classificar a pergunta
- Cadeias específicas: roteiro, logística, info local
- Pinecone para base vetorial com RAG
- LLM da Groq para gerar respostas


Aluna: Mariana Santos Rangel - SAEG


## 1. Setup: Instalação e Imports

In [1]:
from langchain_community.vectorstores import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
import numpy as np
import pathlib
import json
import os
from typing import List, Dict
from langchain_pinecone import Pinecone as PineconeVectorStore
import time
from unidecode import unidecode



/home/mariana/venv310/lib/python3.10/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_pinecone.vectorstores import Pinecone, PineconeVectorStore


## 2. Configuração de Diretórios e Caminhos

In [ ]:
#leitura das chaves
load_dotenv("diretorio/.env", override=True)

# Caminho do dataset turístico
data_path = "diretorio/dados_turisticos.jsonl"

# Diretório para salvar artefatos do projeto (textos + vetores)
art_dir = pathlib.Path("artifacts")
art_dir.mkdir(exist_ok=True)

# Caminhos de saída
out_texts = art_dir / "docs.jsonl"
out_vec = art_dir / "vectors.npy"

# Campos obrigatórios do JSON
REQUIRED = [
    "id", "city", "type", "name", "description",
    "tags", "location", "hours", "price_range", "safety_tips", "lang"
]

## 3. Funções de Normalização e Conversão

In [3]:
# Leitura do .jsonl
def load_jsonl(path: str) -> List[Dict]:
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line.strip()) for line in f if line.strip()]

In [4]:
# Garante presença dos campos obrigatórios
def normalize(rec: Dict) -> Dict:
    norm = {k: rec.get(k) for k in REQUIRED}
    norm["tags"] = norm.get("tags") or []
    return norm



In [5]:
# Gera documento do LangChain
def to_document(rec: Dict) -> Document:
    tags = ", ".join(rec.get("tags", []))
    content = (
        f"{rec['name']} — {rec['type']} em {rec['city']} ({rec['location']}). "
        f"{rec['description']} | Horários: {rec['hours']} | Preço: {rec['price_range']} | "
        f"Tags: {tags} | Dicas: {rec['safety_tips']}"
    )
    meta = {k: rec[k] for k in rec if k != "city"}
  
    return Document(page_content=content, metadata=meta)

## 4. Chunking (divisão dos documentos)

In [6]:
# Divide os textos longos em partes menores
def chunk_docs(docs: List[Document]) -> List[Document]:
    return RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=80).split_documents(docs)

## 5. Gerador de Embeddings (MiniLM-L6-v2)

In [7]:
def get_embedder():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

## 6. Execução do pipeline

In [8]:
# Executa o pipeline de processamento da base
raw = load_jsonl(data_path)
docs = [to_document(normalize(r)) for r in raw]
chunks = chunk_docs(docs)
embedder = get_embedder()
vectors = np.array(embedder.embed_documents([d.page_content for d in chunks]), dtype="float32")


## 7. Salvando os vetores e textos

In [9]:
# Salva textos com metadados
with open(out_texts, "w", encoding="utf-8") as f:
    for doc in chunks:
        record = {
            "text": doc.page_content,
            "metadata": doc.metadata
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

# Salva vetor numpy
np.save(out_vec, vectors)

## 8. Pinecone: Conexão e Indexação

In [10]:
# Conecta à conta
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = os.getenv("PINECONE_INDEX_NAME")

# Exclua índice antigo se existir
if index_name in [i["name"] for i in pc.list_indexes()]:
    pc.delete_index(index_name)
    time.sleep(2)
# Cria índice se não existir
if index_name not in [i["name"] for i in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Conecta ao índice
index = pc.Index(index_name)
print(f"✅ Conectado ao índice: {index_name}")





✅ Conectado ao índice: ia-turismo


## 9. Indexando os vetores no Pinecone

In [11]:
# Reindexa todos os chunks no Pinecone
ids = [f"doc_{i}" for i in range(len(chunks))]
metadata_list = [doc.metadata for doc in chunks]

# Formata os vetores para o Pinecone
to_upsert = list(zip(ids, vectors.tolist(), metadata_list))

# Envia os dados ao índice
index.upsert(vectors=to_upsert)
print(f"✅ Indexados {len(to_upsert)} vetores.")


✅ Indexados 20 vetores.


## RAG (modelo + Pinecone)

In [12]:

# Conectar e criar índice Pinecone com dimensão 384
llm = ChatGroq(model_name='llama-3.3-70b-versatile', temperature=0)

index_name = "ia-turismo"  


vectorstore = PineconeVectorStore.from_documents(
    documents=chunks,
    embedding=embedder,
    index_name=index_name
)

retriever = vectorstore.as_retriever()
chains = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)


## 11. Templates

In [13]:
# Template para roteiro turístico
template_roteiro = """Você é um guia de turismo inteligente.
Com base nas informações abaixo, gere um roteiro de viagem personalizado.
.Se faltar dado, diga que não encontrou no contexto.
Contexto: {context}
Pergunta: {question}
Roteiro:"""

# Template para informações logísticas
template_logistica = """Você é um assistente de viagem. Com base nas informações abaixo, responda dúvidas sobre transporte, acomodações e deslocamentos.
Contexto: {context}
Pergunta: {question}
Resposta:"""

# Template para informações locais (restaurantes, horários, etc.)
template_info_local = """Você é um guia local. Responda à pergunta com base nas informações abaixo.
Contexto: {context}
Pergunta: {question}
Resposta:"""


## 12. Cadeias Especializadas (Chains)

In [14]:
def build_chain(template):
    prompt = PromptTemplate.from_template(template)
    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

# Dicionário de cadeias disponíveis
chains = {
    "roteiro-viagem": build_chain(template_roteiro),
    "logistica-transporte": build_chain(template_logistica),
    "info-local": build_chain(template_info_local),
}

## 13. Roteador Simples com Classificação por Palavra-chave

In [15]:
def classify_question(user_query: str) -> str:
    query = user_query.lower()

    if any(kw in query for kw in ["roteiro", "itinerário", "viagem", "dias"]):
        return "roteiro-viagem"
    elif any(kw in query for kw in ["metrô", "ônibus", "como chegar", "transporte", "hospedagem", "hotel"]):
        return "logistica-transporte"
    elif any(kw in query for kw in ["restaurante", "funciona", "preço", "horário", "ingresso", "aberto"]):
        return "info-local"
    else:
        return "info-local"  # fallback padrão

## 14. Função Final de Consulta (pergunta do usuário)

In [16]:
def responder_pergunta(pergunta: str):
    categoria = classify_question(pergunta)
    resposta = chains[categoria].run(pergunta)
    return resposta



In [19]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Título
display(Markdown("## ✨ IA de Turismo"))
display(Markdown("Peça roteiros, sugestões de passeios, restaurantes, eventos e mais. Ex: *Roteiro para Amsterdã de 3 dias.*"))

# Campo de pergunta
pergunta_input = widgets.Text(
    value="Roteiro para Amsterdã",
    placeholder="Digite sua pergunta aqui...",
    layout=widgets.Layout(width='100%')
)

# Botão
botao = widgets.Button(
    description="Gerar Resposta ✈️",
    button_style="success"
)

# Área de saída
saida = widgets.Output()

# Função de resposta
def gerar_resposta(b):
    with saida:
        clear_output()
        pergunta = pergunta_input.value.strip()
        if not pergunta:
            display(Markdown("⚠️ *Digite uma pergunta.*"))
            return
        try:
            resposta = responder_pergunta(pergunta)
            resposta_formatada = resposta.get("result") if isinstance(resposta, dict) else resposta
            display(Markdown(f"### 📩 **Pergunta:** {pergunta}"))
            display(Markdown(f"### 🧭 **Resposta:**\n\n{resposta_formatada}"))
        except Exception as e:
            display(Markdown(f"❌ Erro: `{e}`"))

botao.on_click(gerar_resposta)

# Exibir interface (⚠️ Não colocar como última linha!)
display(pergunta_input, botao, saida)

# Evita duplicação: adicione uma string “fantasma” no final
""


## ✨ IA de Turismo

Peça roteiros, sugestões de passeios, restaurantes, eventos e mais. Ex: *Roteiro para Amsterdã de 3 dias.*

Text(value='Roteiro para Amsterdã', layout=Layout(width='100%'), placeholder='Digite sua pergunta aqui...')

Button(button_style='success', description='Gerar Resposta ✈️', style=ButtonStyle())

Output()

''